In [1]:
#ProstateX final 

#Andrii Trokoz



'''
Importing all the necessary libraries and frameworks in order to start image processing 
'''
import sys
import SimpleITK as sitk
import matplotlib.pyplot as plt
%matplotlib inline 
import numpy as np
import os
import fnmatch
import shutil
from ipywidgets import interact, fixed
import pandas as pd
import warnings


def myshow(image_arr,slice_n):
    plt.imshow(image_arr[slice_n,:,:],cmap='gray')
    plt.show()

In [2]:
#reading .csv in order to get all of necessary coordinates for cropping 
data_image = pd.read_csv('C://Image Processing/ProstateX-TestLesionInformation/ProstateX-TrainingLesionInformationv2/ProstateX-Images-Train.csv')



In [3]:
#Extracting the names of each image for the further processing from Data Frame, we need this to match names of files with pos coordinates
data_image.loc[:,'match'] = '' #creates new column where we`ll put the matching names 

for i in range(len(data_image)):  #names for further matching with images` names
    data_image.loc[i,'match'] = data_image.loc[i,'DCMSerDescr'] + '_' + data_image.loc[i,'ProxID'].replace('-','_') + '.nii.gz'



In [4]:
'''
Creating separate DataFrames for each of modalites. 

It is going to be: t2_df, adc_df, bval_df and ktrans_df

'''



#creating lists for further processing, renaming and removing the duplicates in the directory  
t2_dir=os.listdir('C://Image Processing/ProstateX/resampled/')
t2_match = [i for i in t2_dir if 't2' in i ]

def make_match_list(directory, matching_value): 
    #generating a list of matching names
    match=[i for i in directory if matching_value in i]
    for i in range(len(match)): 
        if match[i].split('_')[-1][0] !='0':
            match[i]=match[i][:-12] + '.nii.gz'
    
    match_list=[]
    for i in match: 
        if i not in match_list:
            match_list.append(i)
            
    return match_list

#macth lists
t2_list = make_match_list(t2_dir,'t2')

adc_list = make_match_list(t2_dir,'ADC')

bval_list = make_match_list(t2_dir, 'BVAL')



#matching dataframes

#Creating a DataFrame with names of files 

t2_df = data_image[data_image['match'].isin(t2_list)].copy() #use .copy() to overcome SettingWithCopyWarning issue
t2_df.index = range(len(t2_df))

adc_df = data_image[data_image['match'].isin(adc_list)].copy()
adc_df.index = range(len(adc_df))

bval_df = data_image[data_image['match'].isin(bval_list)].copy()
bval_df.index = range(len(bval_df))

In [5]:
#to match all of the files with pos coordinates, we have to assign names to all of our 'match' names
#here we are defining a function to assign names to instances that have different DCMSerNum, but the same name 
#these names are the same as in our files` directory
#function design specifically for this project, cause names of columns in the dataframes and indexing are only suitable for 
#ProstateX data
def rename_missed_match(data):
   #data=corresponding dataframe
    try:
        for i in range(len(data)):
        
            if data.loc[i,'match']==data.loc[i+1,'match'] and data.loc[i,'DCMSerNum'] != data.loc[i+1,'DCMSerNum']:
            
                if len(str(data.loc[i,'DCMSerNum'])) == 1:
                    if len(str(data.loc[i+1,'DCMSerNum'])) != 2:
                        data.loc[i,'match'] = data.loc[i,'match'].strip('.nii.gz')+'_s00'+str(data.loc[i,'DCMSerNum'])+'.nii.gz'
                        data.loc[i+1,'match'] = data.loc[i+1,'match'].strip('.nii.gz')+'_s00'+str(data.loc[i+1,'DCMSerNum'])+'.nii.gz'
                    else: 
                        data.loc[i,'match'] = data.loc[i,'match'].strip('.nii.gz')+'_s00'+str(data.loc[i,'DCMSerNum'])+'.nii.gz'
                        data.loc[i+1,'match'] = data.loc[i+1,'match'].strip('.nii.gz')+'_s0'+str(data.loc[i+1,'DCMSerNum'])+'.nii.gz'
            
                elif len(str(data.loc[i,'DCMSerNum'])) == 2:
                    if len(str(data.loc[i+1,'DCMSerNum'])) != 2:
                        data.loc[i,'match'] = data.loc[i,'match'].strip('.nii.gz')+'_s0'+str(data.loc[i,'DCMSerNum'])+'.nii.gz'
                        data.loc[i+1,'match'] = data.loc[i+1,'match'].strip('.nii.gz')+'_s00'+str(data.loc[i+1,'DCMSerNum'])+'.nii.gz'
                    else:
                        data.loc[i,'match'] = data.loc[i,'match'].strip('.nii.gz')+'_s0'+str(data.loc[i,'DCMSerNum'])+'.nii.gz'
                        data.loc[i+1,'match'] = data.loc[i,'match'].strip('.nii.gz')+'_s0'+str(data.loc[i,'DCMSerNum'])+'.nii.gz'
    except: KeyError


rename_missed_match(t2_df)  #applying functions
rename_missed_match(adc_df)
rename_missed_match(bval_df)




In [ ]:
#Making new column with physical coordinates of the findings 
#In order to do that, we should first read image, than get pos and finally transfer physical point to index

#first of all we should match ProxID and read image 

#for reading image we have to undestand which image to read, get its path and match it with correct pos

#find ProxID -- find filename -- match proxid with filename -- find pos -- read image -- transform image pos to index

def phys_to_index_col(data, folder):
    directory = os.listdir(folder)
    for i in directory: 
        for k in data['ProxID']:
            if k.split('-')[-1] in i: 
                pos=[float(i) for i in (data['pos'].loc[data['ProxID']==k][0]).split(' ')]
                im = sitk.ReadImage(folder+i)
                data['index'].loc[data['ProxID']==k]=im.TransformPhysicalPointToIndex(pos)



In [20]:
#t2_df['coordinates'] = ''
#t2_dir = resampled filed
for i in range(len(t2_df)):
    for image in t2_dir:
        if t2_df.loc[i,'match'] ==image:
            try:
                pos=[float(i) for i in t2_df.loc[i,'pos'].split(' ')]
            except: ValueError
            print(pos, '\tare coordinates for\t'+t2_df.loc[i,'match'])
            #print(t2_df.loc[i,'match'])

[25.7457, 31.8707, -38.511] 	are coordinates for	t2_tse_tra_ProstateX_0000.nii.gz
[-40.5367071921656, 29.320722668457, -16.7076690704346] 	are coordinates for	t2_tse_tra_ProstateX_0001_s006.nii.gz
[-40.5367071921656, 29.320722668457, -16.7076690704346] 	are coordinates for	t2_tse_tra_ProstateX_0001_s010.nii.gz
[-2.058, 38.6752, -34.6104] 	are coordinates for	t2_tse_tra_ProstateX_0002.nii.gz
[-27.0102, 41.5467, -26.0469] 	are coordinates for	t2_tse_tra_ProstateX_0002.nii.gz
[22.1495, 31.2717, -2.45933] 	are coordinates for	t2_tse_tra_ProstateX_0003.nii.gz
[-21.2871, 19.3995, 19.7429] 	are coordinates for	t2_tse_tra_ProstateX_0003.nii.gz
[-7.69665, 3.64226, 23.1659] 	are coordinates for	t2_tse_tra_ProstateX_0004.nii.gz
[-22.0892639160156, 25.4668045043945, 22.8791542053223] 	are coordinates for	t2_tse_tra_ProstateX_0005.nii.gz
[-14.5174331665039, 49.4428329467773, 20.7815246582031] 	are coordinates for	t2_tse_tra_ProstateX_0005.nii.gz
[-38.6276, 42.2781, 21.4084] 	are coordinates for	t2_

[-15.1627, 46.2212, 22.8244] 	are coordinates for	t2_tse_tra_ProstateX_0064.nii.gz
[2.03432, 30.3762, -21.4596] 	are coordinates for	t2_tse_tra_ProstateX_0065.nii.gz
[-21.0782, 9.84683, -28.2478] 	are coordinates for	t2_tse_tra_ProstateX_0065.nii.gz
[-43.1018, 23.1527, -14.4811] 	are coordinates for	t2_tse_tra_ProstateX_0066.nii.gz
[18.6239, 25.4505, 14.9875] 	are coordinates for	t2_tse_tra_ProstateX_0067.nii.gz
[-11.5378, 44.1525, 22.7649] 	are coordinates for	t2_tse_tra_ProstateX_0067.nii.gz
[27.2983, 36.0017, 16.8409] 	are coordinates for	t2_tse_tra_ProstateX_0067.nii.gz
[-22.0582, 39.112, -11.8622] 	are coordinates for	t2_tse_tra_ProstateX_0068.nii.gz
[-32.9206, 11.02, 7.73917] 	are coordinates for	t2_tse_tra_ProstateX_0068.nii.gz
[-8.98097, 42.0751, -9.31191] 	are coordinates for	t2_tse_tra_ProstateX_0068.nii.gz
[-13.9353, 28.1827, -29.5937] 	are coordinates for	t2_tse_tra_ProstateX_0069.nii.gz
[-6.54956, 20.5497, -37.4393] 	are coordinates for	t2_tse_tra_ProstateX_0070.nii.gz
[-5

[31.8466, 21.8933, -8.14504] 	are coordinates for	t2_tse_tra_ProstateX_0121.nii.gz
[31.8466, 21.8933, -8.14504] 	are coordinates for	t2_tse_tra_ProstateX_0122_s012.nii.gz
[-34.0512, 14.0026, -42.5558] 	are coordinates for	t2_tse_tra_ProstateX_0122_s007.nii.gz
[20.8643, 30.9437, 29.9455] 	are coordinates for	t2_tse_tra_ProstateX_0123_s005.nii.gz
[20.8643, 30.9437, 29.9455] 	are coordinates for	t2_tse_tra_ProstateX_0123_s010.nii.gz
[-2.33687, 50.5183, 61.0913] 	are coordinates for	t2_tse_tra_ProstateX_0124.nii.gz
[-37.3152, 28.8733, -27.9854] 	are coordinates for	t2_tse_tra_ProstateX_0125_s006.nii.gz
[-37.3152, 28.8733, -27.9854] 	are coordinates for	t2_tse_tra_ProstateX_0125_s008.nii.gz
[13.6285, 51.202, 6.47909] 	are coordinates for	t2_tse_tra_ProstateX_0126_s004.nii.gz
[-0.00813, 12.961, 3.0117] 	are coordinates for	t2_tse_tra_ProstateX_0126_s006.nii.gz
[-2.96897, 54.8394, -35.4997] 	are coordinates for	t2_tse_tra_ProstateX_0127.nii.gz
[-10.8589, 36.4117, -4.54883] 	are coordinates fo

[3.961669921875, 10.2980422973633, 19.8878326416016] 	are coordinates for	t2_tse_tra_ProstateX_0172.nii.gz
[-7.23110961914062, 43.0895080566406, 37.3951797485352] 	are coordinates for	t2_tse_tra_ProstateX_0172.nii.gz
[-19.9140532625499, 31.4812375854275, 41.345622424212] 	are coordinates for	t2_tse_tra_ProstateX_0172.nii.gz
[-19.6271, 40.7648, -22.8634] 	are coordinates for	t2_tse_tra_ProstateX_0173_s005.nii.gz
[-18.7971, 46.6287, -42.0199] 	are coordinates for	t2_tse_tra_ProstateX_0173_s009.nii.gz
[-19.6271, 40.7648, -22.8634] 	are coordinates for	t2_tse_tra_ProstateX_0173_s009.nii.gz
[-18.7971, 46.6287, -42.0199] 	are coordinates for	t2_tse_tra_ProstateX_0173_s057.nii.gz
[9.31972, 11.9806, 2.38443] 	are coordinates for	t2_tse_tra_ProstateX_0174.nii.gz
[21.0113, 20.9077, -25.4179] 	are coordinates for	t2_tse_tra_ProstateX_0175.nii.gz
[21.851, 36.4468, -28.0461] 	are coordinates for	t2_tse_tra_ProstateX_0175.nii.gz
[26.9851, 36.5301, 15.0113] 	are coordinates for	t2_tse_tra_ProstateX_0

In [ ]:
a=[float(i) for i in (data_all['pos'].loc[data['ProxID']=='ProstateX-0000'][0]).split(' ')]

data_new['ProxID'][1]
data_new.loc[data_new['ProxID']=='ProstateX-0002'].index[0]

a

In [ ]:
data_all['pos'].loc[data['ProxID']=='ProstateX-0191']
directory

In [ ]:
for i in directory:
    for k in data_all['ProxID']:
        if k.split('-')[-1] in i: 
            print(k)
            pos=[float(i) for i in (data_all['pos'].loc[data_all['ProxID']==k][data_all.loc[data['ProxID']==k].index[0]]).split(' ')]
            print(pos)


In [ ]:
extract = sitk.ExtractImageFilter()
extract.SetSize([64,64,0])
extract.SetIndex([153-20, 166-32, 12])
extracted = extract.Execute(im)

extr_arr = sitk.GetArrayFromImage(extracted)
plt.imshow(extr_arr,cmap='gray')

In [ ]:
extract = sitk.ExtractImageFilter()
extract.SetSize([64,64,0])
extract.SetIndex([203-20, 168-32, 9])
extracted = extract.Execute(im)

extr_arr = sitk.GetArrayFromImage(extracted)
plt.imshow(extr_arr,cmap='gray')



In [ ]:
im = sitk.ReadImage('C://Image Processing/ProstateX/t2/t2_tse_tra_ProstateX_0002.nii')
im_arr = sitk.GetArrayFromImage(im)
im_sl_9 = im_arr[9,:,:]
im_sl_9 = sitk.GetImageFromArray(im_sl_9)

extract = sitk.ExtractImageFilter()
extract.SetSize([32,32])
extract.SetIndex([203-20, 168-32])
extracted_image = extract.Execute(im_sl_9)


extrcated_arr = sitk.GetArrayFromImage(extracted_image)

plt.imshow(extrcated_arr, cmap = 'gray')

In [ ]:
myshow(im_arr,9)

In [ ]:
pos = [-27.0102, 41.5467, -26.0469]
ijk = im.TransformPhysicalPointToIndex(pos)

print(ijk)

In [ ]:
im = sitk.ReadImage('C://Image Processing/ProstateX/resampled/ep2d_diff_tra_DYNDIST_ADC_ProstateX_0000.nii')
im_arr = sitk.GetArrayFromImage(im)

im_sl_8 = im_arr[9,:,:]
im_sl_8 = sitk.GetImageFromArray(im_sl_8)

extract = sitk.ExtractImageFilter()
extract.SetSize([36,36])
extract.SetIndex([36, 72 ])
extracted_image = extract.Execute(im_sl_8)


extrcated_arr = sitk.GetArrayFromImage(extracted_image)

plt.imshow(extrcated_arr, cmap = 'gray')

In [ ]:
myshow(im_arr,9)

In [ ]:
data_all.loc[data_all['ProxID']=='ProstateX-0002']